### Wstępny benchmark podstawowego modelu na danych dev oraz subset

In [125]:
import pandas as pd
import os
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio
import torch
from peft import PeftModel, PeftConfig
from datasets import Features, Value, Audio, load_dataset, Dataset
import kagglehub
from jiwer import wer,cer
from tqdm import tqdm
import numpy as np

In [ ]:
dev_manifest = pd.read_csv("piper_out/dev_manifest.csv", sep = "|")
train_manifest = pd.read_csv("piper_out/train_manifest.csv", sep = "|")
test_manifest = pd.read_csv("piper_out/test_manifest.csv", sep = "|")

In [102]:
audio_files = dev_manifest["wav_path"]
references = dev_manifest["text"].tolist()

In [143]:
with open("leki_nom.txt", "r", encoding="utf-8") as f:
    leki = f.read().splitlines()

In [40]:
download_dir = "C:/Users/Admin/Downloads/Posts.csv"

path = kagglehub.model_download(
    "msxksm/whisper-medium-medical-pl/transformers/default"
)

In [ ]:
checkpoint = "natural_anonym_synth"
SAMPLING_RATE = 16000
language = "pl"
task = "transcribe"
peft_config = PeftConfig.from_pretrained(checkpoint) 
model = WhisperForConditionalGeneration.from_pretrained(peft_config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, checkpoint)
merged_model = model.merge_and_unload()
processor = WhisperProcessor.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)

In [118]:
def make_dataset(audio_folder, audio_files, references):
    examples = []
    for audio, ref in zip(audio_files, references):
        filename = os.path.join(audio_folder, audio)
        waveform, sr = torchaudio.load(filename)
        if sr != SAMPLING_RATE:
            waveform = torchaudio.transforms.Resample(sr, SAMPLING_RATE)(waveform)
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)
        
        examples.append({"waveform": waveform, "phrase": ref})
    return examples

In [ ]:
dataset = make_dataset("dev_noisy", audio_files, references)

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(


In [107]:
def transcribe_example(example):
    input_features = processor(
        example["waveform"].squeeze(0), 
        sampling_rate=SAMPLING_RATE,
        return_tensors="pt"
    ).input_features

    with torch.no_grad():
        predicted_ids = merged_model.generate(
            input_features, 
            forced_decoder_ids=forced_decoder_ids
        )[0]

    transcription = processor.decode(predicted_ids, skip_special_tokens=True)
    return transcription

In [110]:
hypotheses = []
references = []

for ex in tqdm(dataset, desc="Transcribing"):
    transcription = transcribe_example(ex)
    hypotheses.append(transcription)
    references.append(ex["phrase"])

Transcribing:   0%|          | 0/146 [00:00<?, ?it/s]

Transcribing: 100%|██████████| 146/146 [25:05<00:00, 10.31s/it]


In [111]:
wer_score = wer(references, hypotheses)
cer_score = cer(references, hypotheses)

print(f"Subset WER: {wer_score:.4f}")
print(f"Subset CER: {cer_score:.4f}")

Subset WER: 0.9536
Subset CER: 0.5920


In [112]:
for ref, hyp in zip(references, hypotheses):
    print(f"Reference: {ref}\tHypothesis: {hyp}")

Reference: Powinnaś spróbować kuracji Ibupromem.	Hypothesis: Po winu nie spróbować kuracji w upromem.
Reference: Najlepiej w twoim przypadku sprawdzi się Ibuprom.	Hypothesis: Najlepiej w twoim przypadku sprawdzi się IBU-PRO.
Reference: Powinnaś spróbować kuracji Metafenem.	Hypothesis: Pożynne spróbować kuracji metafenem.
Reference: Możesz kupić w aptece Metafen, powinien pomóc.	Hypothesis: Możesz kupić w aptycy metafem, powinien pomóc.
Reference: Najlepiej w twoim przypadku sprawdzi się Nurofen Forte.	Hypothesis: Najlepiej w twoim przypadku sprawdzi się nurowenworte.
Reference: Zalecam Nurofen Forte, bo dobrze działa na ból i stan zapalny.	Hypothesis: Zalecam norowę Fortem, bo dobrze działa nabór i stan zapalny.
Reference: Przepiszę ci Dolgit, bo zwykle dobrze działa.	Hypothesis: Przepiszę ci doldit w osłupę, pobudzę węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węzeł węz

In [147]:
folder = "subset"
files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

In [148]:
subset_dataset = make_dataset("subset", files, leki)

In [149]:
hypotheses = []
references = []

for ex in tqdm(subset_dataset, desc="Transcribing"):
    transcription = transcribe_example(ex)
    hypotheses.append(transcription)
    references.append(ex["phrase"])

Transcribing: 100%|██████████| 75/75 [10:27<00:00,  8.36s/it]


In [150]:
cer_score = cer(references, hypotheses)

print(f"Subset CER: {cer_score:.4f}")

Subset CER: 1.0990


In [151]:
for ref, hyp in zip(references, hypotheses):
    print(f"Reference: {ref}\tHypothesis: {hyp}")

Reference: Actarosin	Hypothesis: Ostarosnie.
Reference: ActiFolin	Hypothesis: Abstryj folii.
Reference: Aliness Kwas Foliowy	Hypothesis: Alinis quas foillowy.
Reference: Amlodipina	Hypothesis: Auto do dipina.
Reference: Amoksycylina	Hypothesis: Amoż licejny.
Reference: Amotaks	Hypothesis: Amotax.
Reference: ApoD3	Hypothesis: Apoedektyczna opieka w KG.
Reference: Arterios	Hypothesis: Aktelio.
Reference: Augmentin	Hypothesis: Augunantnie.
Reference: Avamina	Hypothesis: Awa minął.
Reference: Biaron D	Hypothesis: Jak odbiór rąde.
Reference: Bibloc	Hypothesis: PiWC.
Reference: Bisakodyl	Hypothesis: Dli zakończonych leków nie zalecanią.
Reference: Bisocard	Hypothesis: I sotoc.
Reference: Bisoprolol	Hypothesis: Nie zauważyłem.
Reference: Bisoratio	Hypothesis: Wii sora dnia.
Reference: Concor	Hypothesis: Ponco".
Reference: Coronal	Hypothesis: Trona.
Reference: Crestor	Hypothesis: Krystolat w kontrolnym ZWK nie obniża się.
Reference: Crosuvo	Hypothesis: Proszę wą krossofool.
Reference: D-Vitum 